# Evaluate API pipeline

This notebook run an evaluation script based on the Rouge Metric to compute the eficience of the API pre-trained pipeline to generate and translate captions.

In [2]:
predict = False
max_qnt = 5000

## Imports

In [3]:
!pip install evaluate pandas
!pip install transformers torch
!pip install rouge_score

In [1]:
from evaluate import load
import pandas as pd
from transformers import AutoTokenizer 
from transformers import AutoModel
import torch
import nltk
nltk.download("punkt")

/home/mariaeas/ZoIA/research/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mariaeas/ZoIA/research/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package punkt to /home/mariaeas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Build predictions dataset

In [2]:
test_set = "prediction_reference.csv"
test_df = pd.read_csv(test_set)

In [3]:
test_df.shape

(2133, 4)

In [5]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("/home/mariaeas/ZoIA/research/model-250424/image-captioning-output250324", pad_token_id=0)
image_captioner = pipeline("image-to-text", model="/home/mariaeas/ZoIA/research/model-250424/image-captioning-output250324", tokenizer=tokenizer)

/home/mariaeas/ZoIA/research/.venv/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [7]:
image = "parque-infantil.jpg"
image_path = f"./{image}"
prediction = image_captioner(image_path)[0]['generated_text']
prediction

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/mariaeas/ZoIA/research/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


' Olivaisfoto mostra uma menina de aproximadamente dez anos em pé ela está de costas para a câmera e'

In [9]:
df = {
    "image": [],
    "caption": [],
    "prediction": []
}

for i in range(test_df.shape[0]):
    row = test_df.iloc[i]

    image = row['image']
    caption = row['caption']
    image_path = f"images/63k-subset/{image}"
    prediction = image_captioner(image_path)[0]['generated_text']
    print(prediction)

    df['image'].append(image)
    df['caption'].append(caption)
    df['prediction'].append(prediction)

    pd.DataFrame(df).to_csv("out.csv")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/mariaeas/ZoIA/research/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_

In [31]:
#df = pd.DataFrame(df)
df = pd.read_csv("prediction_test_model.csv")

In [32]:
df.head()

,Unnamed: 0,image,caption,prediction
0,0,i-00098484.jpg,capa livro 71 leões título escrito cor amarela...,olivaisfoto mulher costas mochila mão homem co...
1,1,i-00098503.jpg,imagem braille página livro pequeno príncipe m...,olivaisfoto mão segurando livro aberto frase e...
2,2,i-00098515.jpg,foto tabela informações cursos formação oferec...,olivaisimagem fundo branco detalhes azul topo ...
3,3,i-00098481.jpg,coelhinho branco orelhas rosadas usando óculos...,olivaisem fundo branco ilustração gato branco ...
4,4,i-00098508.jpg,professora adriana aluna celinha fantasiadas r...,olivaisa foto mostra cantora apresentadora sab...


In [34]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mariaeas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
import re

In [37]:
sw = stopwords.words('portuguese')
sw.extend(['.', ":", "!", "?", "\\", "/", "*", "-", ",", "#", "@"])
def clean_text(text):
    tokens = nltk.word_tokenize(text.strip(), language="portuguese")
    filtered_tokens = list(filter(lambda w: w not in sw, tokens))

    return re.sub("olivais", "", " ".join(filtered_tokens).lower())

In [38]:
df['caption'] = df['caption'].apply(clean_text)
df['prediction'] = df['prediction'].apply(clean_text)

df.head()

,Unnamed: 0,image,caption,prediction
0,0,i-00098484.jpg,capa livro 71 leões título escrito cor amarela...,foto mulher costas mochila mão homem costas
1,1,i-00098503.jpg,imagem braille página livro pequeno príncipe m...,foto mão segurando livro aberto frase escrita ...
2,2,i-00098515.jpg,foto tabela informações cursos formação oferec...,imagem fundo branco detalhes azul topo imagem ...
3,3,i-00098481.jpg,coelhinho branco orelhas rosadas usando óculos...,em fundo branco ilustração gato branco orelhas...
4,4,i-00098508.jpg,professora adriana aluna celinha fantasiadas r...,a foto mostra cantora apresentadora sabrina sa...


# Calculate metrics

In [33]:
!pip install aac-metrics

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 760.1 kB/s eta 0:00:00 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached scikit_learn-1.4.1.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 538.8 kB/s eta 0:00:0031m3.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 2.9 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached scikit_learn-1.4.1.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.4 MB)
  Created wheel for aac-metrics: filename=aac_metrics-0.5.4-py3-none-any.whl size=81164 sha256=236

In [34]:
!aac-metrics-download

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[2024-04-01 13:47:33,001][aac_metrics.download][INFO] - aac-metrics download started.
[2024-04-01 13:47:33,002][aac_metrics.download][INFO] - Downloading JAR source for 'stanford_nlp' in directory /home/mariaeas/.cache/aac-metrics/stanford_nlp.
100%|██████████████████████████████████████| 5.65M/5.65M [00:01<00:00, 4.10MB/s]
[2024-04-01 13:47:36,643][aac_metrics.download][INFO] - Downloading source for 'meteor-1.5.jar' in directory /home/mariaeas/.cache/aac-metrics/meteor.
100%|██████████████████████████████████████| 6.03M/6.03M [00:01<00:00, 4.99MB/s]
[2024-04-01 13:47:38,909][aac_metrics.download][INFO] - Downloading source for 'data/paraphrase-en.gz' in directory /home/mariaeas/.cache/aac-metrics/meteor.
100%|██████████████████████████████████████| 58.9M/58.9M [00:13<00:00, 4.67MB/s]
[2024-04-01 13:47:54,638][aac_metrics.download][INFO] - Downloading source for 'data/paraphrase-fr.gz' in directory /home/mariaeas/.cache/aac-metrics/meteor.
100%|██████████████████████████████████████| 

In [9]:
from aac_metrics import evaluate

In [39]:
candidates = list(df.prediction)
mult_references = list(map(lambda x: [x], list(df.caption)))
#scores, _ = evaluate(candidates, mult_references)


In [46]:
mult_references[3], candidates[3]

(['coelhinho branco orelhas rosadas usando óculos escuros casaco couro preto mensagem feliz páscoa'],
 'em fundo branco ilustração gato branco orelhas coelho óculos')

## Calculate BLEU

In [47]:
metric_bleu = load('bleu')
result_bleu = metric_bleu.compute(predictions=candidates, references=mult_references)
result_bleu

{'bleu': 0.0015063238597797556,
 'precisions': [0.1828009828009828,
  0.024657026325546904,
  0.005267278809879707,
  0.0008392077878482712],
 'brevity_penalty': 0.12678696204595738,
 'length_ratio': 0.32623797648735303,
 'translation_length': 18315,
 'reference_length': 56140}

## Calculate ROUGE

In [48]:
metric_rouge = load('rouge')
result_rouge = metric_rouge.compute(predictions=candidates, references=mult_references)
result_rouge

{'rouge1': 0.11051715722678206,
 'rouge2': 0.02105177501598126,
 'rougeL': 0.1007692568481812,
 'rougeLsum': 0.10074964040168591}

# Calculate METEOR

In [49]:
meteor = load('meteor')
result_meteor = meteor.compute(predictions=candidates, references=mult_references)
result_meteor

[nltk_data] Downloading package wordnet to /home/mariaeas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/mariaeas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mariaeas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.048888737330145295}

In [80]:
!pip install git+https://github.com/ruotianluo/cider.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/ruotianluo/cider.git to /tmp/pip-req-build-neeg44a9
  Running command git clone --filter=blob:none --quiet https://github.com/ruotianluo/cider.git /tmp/pip-req-build-neeg44a9
  Resolved https://github.com/ruotianluo/cider.git to commit dbb3960165d86202ed3c417b412a000fc8e717f3
ERROR: git+https://github.com/ruotianluo/cider.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


# CIDeR

In [84]:
from aac_metrics import evaluate

cider, _ = evaluate(candidates, mult_references, metrics="cider_d")
float(cider['cider_d'])

0.04983575958883829

## Calculate Cosine Similarity

In [50]:
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=True, padding='max_length', max_len=500)

In [51]:
from torch.nn.functional import cosine_similarity

In [52]:
mean_cosine_similarity = 0
for i in range(df.shape[0]):
    row = df.iloc[i]
    
    sentence1 = str(row['caption'])
    sentence2 = str(row['prediction'])

    # Tokenize and encode sentences
    inputs1 = tokenizer(sentence1, return_tensors='pt', truncation=True, padding=True)
    inputs2 = tokenizer(sentence2, return_tensors='pt', truncation=True, padding=True)

    # Encode sentences
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # Extract embeddings
    embeddings1 = outputs1.last_hidden_state[:, 0, :]
    embeddings2 = outputs2.last_hidden_state[:, 0, :]

    # Calculate cosine similarity
    similarity_score = cosine_similarity(embeddings1, embeddings2).item()
    mean_cosine_similarity += similarity_score
    print(f"file: {row['image']} | similarity = {similarity_score}")

file: i-00098484.jpg | similarity = 0.6935357451438904
file: i-00098503.jpg | similarity = 0.8212555646896362
file: i-00098515.jpg | similarity = 0.6470127105712891
file: i-00098481.jpg | similarity = 0.8148306608200073
file: i-00098508.jpg | similarity = 0.6510591506958008
file: i-00098482.jpg | similarity = 0.5459609031677246
file: i-00098488.jpg | similarity = 0.826174259185791
file: i-00098498.jpg | similarity = 0.7214704751968384
file: i-00098500.jpg | similarity = 0.6997847557067871
file: i-00098504.jpg | similarity = 0.7050526142120361
file: i-00098516.jpg | similarity = 0.7342643737792969
file: i-00098506.jpg | similarity = 0.8402043581008911
file: i-00098518.jpg | similarity = 0.783808171749115
file: i-00098501.jpg | similarity = 0.7299285531044006
file: i-00098509.jpg | similarity = 0.6126794815063477
file: i-00098510.jpg | similarity = 0.8322607278823853
file: i-00098511.jpg | similarity = 0.7719410061836243
file: i-00098494.jpg | similarity = 0.7988409399986267
file: i-0009

In [53]:
mean_cosine_similarity /= df.shape[0]
mean_cosine_similarity

0.7584646409853377

# Metrics Overview

In [89]:
metrics = {
    "metric": ["BLEU1", "BLEU2","BLEU3","BLEU4", "ROUGE1", "ROUGE2", "ROUGEL", "METEOR", "CIDEr-D", "COSINE SIMILARITY"],
    "value": [result_bleu['precisions'][0], result_bleu['precisions'][1],result_bleu['precisions'][2], result_bleu['precisions'][3], result_rouge['rouge1'], result_rouge['rouge2'], result_rouge['rougeL'], result_meteor['meteor'], float(cider['cider_d']), mean_cosine_similarity]
}

df_metrics = pd.DataFrame(metrics)
df_metrics

In [90]:
df_metrics = pd.DataFrame(metrics)
df_metrics

,metric,value
0,BLEU1,0.182801
1,BLEU2,0.024657
2,BLEU3,0.005267
3,BLEU4,0.000839
4,ROUGE1,0.110517
5,ROUGE2,0.021052
6,ROUGEL,0.100769
7,METEOR,0.048889
8,CIDEr-D,0.049836
9,COSINE SIMILARITY,0.758465


In [23]:
df.shape[0]

2133

In [24]:
df_metrics.to_csv("model_metrics.csv")

In [25]:
df.to_csv("prediction_test_model.csv")

In [36]:
tokenizer.pad_token_id

0

In [34]:
prediction

' Olivaisfoto mostra uma menina de aproximadamente dez anos em pé ela está de costas para a câmera e'